## Week 7 Topic Modeling (LDA)

###connect Colab to your Google Drive.

In [1]:
#connect Colab to your Google Drive.
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd

#import csv file and put it into a Pandas dataframe.
hotel=pd.read_csv('/content/gdrive/My Drive/CIS NLP Data Sets/hotel-reviews.csv')


#movie.head()
print (hotel.iloc[:10,:])

#How many columns and rows?
print ("Shape:", hotel.shape)

#Column names?
print ("Column Names",hotel.columns.values)

   User_ID                                        Description  \
0  id10326  The room was kind of clean but had a VERY stro...   
1  id10327  I stayed at the Crown Plaza April -- - April -...   
2  id10328  I booked this hotel through Hotwire at the low...   
3  id10329  Stayed here with husband and sons on the way t...   
4  id10330  My girlfriends and I stayed here to celebrate ...   
5  id10331  We had - rooms. One was very nice and clearly ...   
6  id10332  My husband and I have stayed in this hotel a f...   
7  id10333  My wife & I stayed in this glorious city a whi...   
8  id10334  My boyfriend and I stayed at the Fairmont on a...   
9  id10335  Wonderful staff, great location, but it was de...   

        Browser_Used Device_Used Is_Response  
0               Edge      Mobile   not happy  
1  Internet Explorer      Mobile   not happy  
2            Mozilla      Tablet   not happy  
3   InternetExplorer     Desktop       happy  
4               Edge      Tablet   not happy  
5 

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Pre-processing

In [5]:
#lemmatization (full version)

#lemmatization using PoS tagging
#lemmatizer = nltk.stem.WordNetLemmatizer()
#wordnet_lemmatizer = WordNetLemmatizer()
#from nltk.corpus import wordnet

#def nltk_tag_to_wordnet_tag(nltk_tag):
#    if nltk_tag.startswith('J'):
#        return wordnet.ADJ
#    elif nltk_tag.startswith('V'):
#        return wordnet.VERB
#    elif nltk_tag.startswith('N'):
#        return wordnet.NOUN
#    elif nltk_tag.startswith('R'):
#        return wordnet.ADV
#    else:
#        return None

#def lemmatize_sentence(sentence):
#    #tokenize the sentence and find the POS tag for each token
#    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
#    #tuple of (token, wordnet_tag)
#    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#    lemmatized_sentence = []
#    for word, tag in wordnet_tagged:
#        if tag is None:
#            #if there is no available tag, append the token as is
#            lemmatized_sentence.append(word)
#        else:
#            #else use the tag to lemmatize the token
#            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
#    return " ".join(lemmatized_sentence)


#lemmatization (simplified version)
def lemmatize_text(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

# and then remove stop words and words fewer than 2 characters.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_text(token))
    return result

hotel['Description_lemmatized_processed'] = hotel['Description'].map(preprocess)

print (hotel.iloc[:10,:])
print ("Shape:", hotel.shape)
print ("Column Names",hotel.columns.values)


   User_ID                                        Description  \
0  id10326  The room was kind of clean but had a VERY stro...   
1  id10327  I stayed at the Crown Plaza April -- - April -...   
2  id10328  I booked this hotel through Hotwire at the low...   
3  id10329  Stayed here with husband and sons on the way t...   
4  id10330  My girlfriends and I stayed here to celebrate ...   
5  id10331  We had - rooms. One was very nice and clearly ...   
6  id10332  My husband and I have stayed in this hotel a f...   
7  id10333  My wife & I stayed in this glorious city a whi...   
8  id10334  My boyfriend and I stayed at the Fairmont on a...   
9  id10335  Wonderful staff, great location, but it was de...   

        Browser_Used Device_Used Is_Response  \
0               Edge      Mobile   not happy   
1  Internet Explorer      Mobile   not happy   
2            Mozilla      Tablet   not happy   
3   InternetExplorer     Desktop       happy   
4               Edge      Tablet   not happy

## Create a dictionary from the data set.

In [6]:
#Create a dictionary from 'Description_lemmatized_processed' containing the number of times a word appears 
#in the data set using gensim.corpora.Dictionary.

dictionary = gensim.corpora.Dictionary(hotel['Description_lemmatized_processed'])

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

print (len(dictionary))

0 average
1 better
2 breakfast
3 clean
4 consider
5 dog
6 free
7 fussy
8 generally
9 kind
10 overnight
37068


## Remove too common & context-specific tokens.

In [7]:
#Gensim filter_extremes.

#filter_extremes(no_below=i, no_above=j, keep_n=k) where i,j,k can be integers or fractions.

#Filter out tokens that appear in:
#less than no_below documents (absolute number) or
#more than no_above documents (fraction of total corpus size, not absolute number).
#after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

#Filter out tokens that appear in:
#less than 100 documents.
#more than 0.7 documents.
dictionary.filter_extremes(no_below=500, no_above=0.70)
print (len(dictionary))

752


## Gensim doc2bow to tranform the tokens into vectors.

In [8]:
#Gensim doc2bow (pass the tokenized words to doc2bow and convert those to vectors.)

## Caution: No further preprocessing should be done such as tokenization, lemmatization, and etc before initiating this.

#For each document, check how many words and how many times those words appear.
bow_corpus = [dictionary.doc2bow(doc) for doc in hotel['Description_lemmatized_processed']]
print (len(bow_corpus))
print (bow_corpus[0])

#pretiffy the above result to see how the 1st document in our data has been converted.
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]],bow_doc_0[i][1]))

38932
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2)]
Word 0 ("average") appears 1 time.
Word 1 ("better") appears 1 time.
Word 2 ("breakfast") appears 1 time.
Word 3 ("clean") appears 1 time.
Word 4 ("consider") appears 1 time.
Word 5 ("free") appears 1 time.
Word 6 ("generally") appears 1 time.
Word 7 ("kind") appears 1 time.
Word 8 ("overnight") appears 1 time.
Word 9 ("price") appears 1 time.
Word 10 ("right") appears 1 time.
Word 11 ("smell") appears 1 time.
Word 12 ("stay") appears 2 time.


## Applying TF-IDF 

In [10]:
#create tf-idf model object using models.
#TfidfModel on bow_corpus
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

#Apply transformation to the entire corpus.
corpus_tfidf = tfidf[bow_corpus]
print (len(corpus_tfidf))



38932


In [11]:
#pretiffy the above result to see how the 1st document in our data has been converted.
corpus_tfidf_0=corpus_tfidf[0]
from pprint import pprint
#for doc in corpus_tfidf:
pprint(corpus_tfidf_0)
    


[(0, 0.3400201488467968),
 (1, 0.22547077468001045),
 (2, 0.1280704209901086),
 (3, 0.09861088101941841),
 (4, 0.3316467989276336),
 (5, 0.17798059898323637),
 (6, 0.44290000747535424),
 (7, 0.3251172930602114),
 (8, 0.4304704686805295),
 (9, 0.1751490322634236),
 (10, 0.1940191700197154),
 (11, 0.3232813880437279),
 (12, 0.07588427856990787)]


## Run LDA using TF-IDF

In [12]:
#Run LDA model on the final corpus.

#num_topics: the number of latent topics to be extracted from the corpus.
#id2word: mapping from word ids (integers) to words (strings).
# Some other parameters. See the document explanations for more details.

#lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=dictionary, eta='auto', iterations=100,
#                                             random_state=100, alpha='symmetric', passes=2, workers=4)

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=dictionary
                                             )

In [14]:
import re
for idx, topic in lda_model_tfidf.print_topics(-1, num_words=10):
    #print out topic numbers and keywords.
#    print('Topic: {} Word: {}'.format(idx, topic))
    
    #print out keywords only (without probability)
    key_words_only = " ".join(re.findall("[a-zA-Z]+", topic))
    print ('Topic:',idx,key_words_only)

Topic: 0 francisco walk great wharf union park beach cable nice location
Topic: 1 great pool love wonderful service staff enjoy thank beautiful stay
Topic: 2 good free breakfast walk internet clean nice location great park
Topic: 3 great friendly staff location comfortable service helpful good excellent clean
Topic: 4 bathroom nice small suite shower great area large floor comfortable
Topic: 5 york square time central subway great walk location state manhattan
Topic: 6 check tell desk night go call leave say like work


## Finding optimal number of topics

In [15]:
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt 

def calculate_coherence_values(dictionary, corpus, texts, limit, start=4, step=2):
    '''
    Compute c_v coherence for various number of topics to find the OPTIMAL NUMBER OF TOPICS.

    Parameters:
              dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              limit : Max num of topics
              start: starting number of topics
              step=n : increase the number by n
    '''

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model =  gensim.models.LdaMulticore(corpus_tfidf, num_topics=num_topics, id2word=dictionary
                                             )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=hotel['Description_lemmatized_processed'], dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

# Can take a while to see the outputs.
model_list, coherence_values = calculate_coherence_values(dictionary=dictionary, corpus=corpus_tfidf, texts=hotel['Description_lemmatized_processed'], start=4, limit=10, step=2)

# Show graph
limit=10; start=4; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()


NameError: ignored

## Find dominant topics for each doc.

In [ ]:
#Find the dominant topic in each document
list_in_list=[]

#for the first 4 documents.
doc_topics = lda_model_tfidf.get_document_topics(corpus_tfidf[:5])
print('document topics: ', doc_topics)

for each_doc in doc_topics:
    
    #get the assigned topics and list them in terms of their highest to lowest probability.
    top_topic=sorted(each_doc, key=lambda x: x[1], reverse=True)[0]

    #If you want to see the 2nd dominant topic:
    #top_topic_2=sorted(each_doc, key=lambda x: x[1], reverse=True)[1]
    
    print ("highest",top_topic)
    
    #select topic number & its probability.
    top_topic_num=top_topic[0]
    top_topic_prob=top_topic[1]

    #make a dataframe and fill out within each column.        
    list=[top_topic_num,top_topic_prob]
    list_in_list.extend([list])
    #print (list_in_list)
#create pandas dataframe with column names.
df = pd.DataFrame(list_in_list, columns = ['Dominant Topic', 'Probability']) 
pd.set_option("display.max_rows", None, "display.max_columns", None)
print (df)